In [1]:
import pandas as pd
import numpy as np
import math
import sqlite3
import os

𝐴𝐶𝑆=Σ(𝐶𝐶𝑖+𝑅𝐶𝑖)×𝐶𝑅𝐹(𝑖𝑟,𝑅)+𝑂&𝑀𝑖)

𝐴𝐶𝑆𝑎𝑑𝑗=[(𝐶𝐶𝑝𝑣+𝐶𝐶𝑏𝑎𝑡)×Δ+𝐶𝐶𝐷𝐺+𝑅𝐶𝑏𝑎𝑡+𝑅𝐶𝐷𝐺]×𝐶𝑅𝐹(𝑖𝑟,𝑅)+𝑂&𝑀𝑝𝑣+𝑂&𝑀𝑏𝑎𝑡+𝑂&𝑀𝐷𝐺

## Indicadores financieros

𝐶𝑅𝐹(𝑖𝑟,𝑅)=𝑖𝑟×(1+𝑖𝑟)^𝑅/(1+𝑖𝑟)^𝑅−1

In [2]:
def conexion_bd(sql=None, update=True):
    
    sqlite3.register_adapter(np.int64, lambda val: int(val))
    sqlite3.register_adapter(np.int32, lambda val: int(val))

    con = sqlite3.connect(os.getcwd() + os.sep + "result_op.db")
    
    
    if update == True:
        cur = con.cursor()
        cur.execute(sql)
        con.commit()
        df=None
    else:
        df= pd.read_sql_query(sql, con=con)
    con.close()
    
    return df
    


In [3]:
parametros = conexion_bd("SELECT * FROM parametros WHERE optimizacion = 'optimal'",  update=False)
resultados = conexion_bd("SELECT * FROM resultados ", update=False)

In [4]:
trm = 3736.91

id_simulacion = parametros.loc[0,'id_simulacion']
n_pv = parametros.loc[0,'n_pv']
n_dg = parametros.loc[0,'n_dg']
p_bat = parametros.loc[0,'p_bat']
p_dg = parametros.loc[0,'p_dg']
ens = parametros.loc[0,'ens_result']

c_pv = parametros.loc[0,'cost_pv']
c_bat = parametros.loc[0,'cost_bat']
c_dg = parametros.loc[0,'cost_dg']
c_ens = parametros.loc[0,'cost_pens']

In [5]:
resultado_id = resultados[resultados['id_simulacion']==id_simulacion].reset_index(drop=True)

In [6]:
resultado_id['total_dg'] = resultado_id['energia_Dg'] + resultado_id['p_bat_dg']
resultado_id['total_pv'] = resultado_id['energia_PV'] + resultado_id['p_bat_pv']

In [7]:
ei = round(resultado_id['total_dg'].sum(),2)
et = round(resultado_id['total_dg'].sum(),2)

et_pv = round(resultado_id['total_pv'].sum(),2)

load = round(resultado_id['load'].sum(),2)

et_bat = round(resultado_id['energia_descarga_bateria'].sum(),2)

cost_e_dg = (et*c_dg)/trm
cost_e_pv = (et_pv*c_pv)/trm
cost_e_bat =( et_bat*c_bat)/trm
cost_e_ens =( ens*c_ens)/trm


In [8]:
et/load


0.3922102067762728

In [9]:
financiero = {'R':20, 'ir':0.0808, 'cpv_ins' : 5605.365 ,'cbat_ins' : 539983.495,'cdg_ins' : 7627407.001,  'npv' : n_pv ,'ndg' : n_dg ,'ppv_stc': 300,'ebcell_nom': p_bat,
'pdg_rate': p_dg, 'li_bat':10,'li_dg':10,'ybat': 0.7,'ydg': 0.7, 'ipp_o': 74.37,'ipp_actual' : 129.23, 'cec': 0.0974,'ei': ei,'et': et,'pami': 8789 ,
'cel':0.0005,'plim':79900, 'p_load':load,'ens':ens, 'factor_pv':0.01,'factor_bat':0.02}

In [10]:
#i_n = 2 # nominal interest rate
#i_f = 1 # expected annual inflation rate respectively

R = financiero['R'] # the life time of the project
ir = financiero['ir']  #  (i_n - i_f) / (1 + i_f) # Tomado de otro estudio

crf = round(ir*((1+ir)**R) /((1+ir)**R - 1),2)  # The capital recovery factor is calculated by

In [11]:
crf

0.1

## Calculos de Inversión

𝐶𝐶𝑝𝑣=𝑐𝑝𝑣×𝑁𝑝𝑣×𝑃𝑝𝑣𝑠𝑡𝑐
𝐶𝐶𝑏𝑎𝑡=𝑐𝑏𝑎𝑡×𝐸𝑏𝑐𝑒𝑙𝑙_𝑛𝑜𝑚
𝐶𝐶𝐷𝐺=𝑐𝐷𝐺×𝑁𝐷𝐺×𝑤𝐷𝐺

In [12]:
cpv_ins = financiero['cpv_ins'] # costo de PV kWh instalado
cbat_ins = financiero['cbat_ins'] # costo Battery de kWh instalado
cdg_ins = financiero['cdg_ins'] # costo diesel de kWh instalado

npv = financiero['npv'] # Número de paneles fotovoltaicos
ndg = financiero['ndg'] # Número planta diesel
ppv_stc = financiero['ppv_stc']# Capacidad nominal paneles
ebcell_nom = financiero['ebcell_nom'] # Capacidad de la batería
pdg_rate = financiero['pdg_rate'] # Capacidad nominal diesel

ccpv = round(cpv_ins*npv*ppv_stc,4)
ccbat = round(cbat_ins*ebcell_nom,4)
ccdg = round(cdg_ins*ndg*pdg_rate,4)

## Costos de reemplazo

𝑅𝐶𝑏𝑎𝑡=𝛾𝑏𝑎𝑡×𝐶𝐶𝑏𝑎𝑡×𝐾𝑏𝑎𝑡(𝑖𝑟,𝐿𝑝𝑣,𝑦𝑖)
𝑅𝐶𝐷𝐺=𝛾𝐷𝐺×𝐶𝐶𝐷𝐺×𝐾𝐷𝐺(𝑖𝑟,𝐿𝐷𝐺,𝑦𝑖)

𝐾𝑖(𝑖𝑟,𝐿𝑖,𝑦𝑖)=Σ1(1+𝑖𝑟)𝑛×𝐿𝑖

In [13]:
def calcular_ki(R, li, ir):
    """
    Para cálcular single payment present worth
    """
    yi_replacements = math.floor(R/li)
    values_to_sum = []
    for i in range(1,yi_replacements+1):
        x = (1)/((1+ir)**(i*li))
        values_to_sum.append(x)
        
    return sum(values_to_sum)

kbat = round(calcular_ki(R=financiero['R'], li=financiero['li_bat'], ir=financiero['ir']),4)  # single payment present worth battery
kdg = round(calcular_ki(R=financiero['R'], li=financiero['li_dg'], ir=financiero['ir']),4) # single payment present worth diesel

In [14]:
ybat = financiero['ybat'] #are de-rate factors of the initial capital cost invested 
ydg = financiero['ydg'] #are de-rate factors of the initial capital cost invested

rc_bat = round(ybat*ccbat*kbat,4)
rc_dg = round(ydg*ccdg*kdg,4)

## Calculos de operación y Mantto

### Operación y mantto del PV y BATTERY


In [15]:
factor_pv = financiero['factor_pv'] # Factor de la inversión inicial
factor_bat = financiero['factor_bat'] # Factor de la inversión inicial

oym_pv = factor_pv*ccpv 
oym_bat = factor_bat*ccbat

### Operación y mantto del DIESEL

##### Costo de Combustible (CC)

In [16]:
ipp_o = financiero['ipp_o']
ipp_actual = financiero['ipp_actual']

cec = financiero['cec'] #Consumo especifíco de combustible 0.0974 gal/kWh (capacidad <= 100 kW)
ei = financiero['ei'] #Energía entregada al Sistema de Distribución por el generador i 
et = financiero['et'] #Energía total entregada al Sistema de Distribución

pami = financiero['pami'] #Precio promedio del combustible para la planta de abasto más cercana al generador i en el mes m ($/gal).
tmi = pami*0.1
calm = 82.14*(ipp_actual/ipp_o) # Costo de almacenamiento de combustible en el mes m ($/gal)
pci = pami + tmi + calm  # Precio del galón  en el sitio para el generador i

cc = (1/et)*(cec*pci*ei) # Costo de Combustible (CC)

##### Costo de Lubricante (CL)

In [17]:
cel = financiero['cel'] # Consumo Específico de Lubricante 0,00050 gal/kWh para plantas de capacidad <= 2.000 Kw
plim = financiero['plim'] #Precio del Galón de lubricante en el sitio para el generador i en el mes m ($/gal). el precio del lubricante se determinará con base en los precios promedio del mercado.
cl = (1/et)*(cel*(plim+tmi)*ei)

##### Gastos Administrativos (CA)

In [18]:
cam = 0.1*(cc+cl) # Los gastos administrativos para el mes m corresponderán al 10% de la suma de los costos por consumo de combustible y lubricante,

In [19]:
oym_dg = (cam + cc + cl)*ei 

In [20]:
oym_dg

15669183.320112605

#  Annual  System  Cost  (ASC)

In [21]:
incentivo = 0.9038

In [22]:
asc = (((ccpv+ccbat+ccdg)+(rc_bat+rc_dg))*crf + (oym_dg + oym_pv + oym_bat))/trm
asc_incentivo = ((((ccpv+ccbat)*incentivo+ccdg)+(rc_bat+rc_dg))*crf + (oym_dg + oym_pv + oym_bat))/trm

## Cálculo del incentivo


In [153]:
T1 = 15
T2 = 10
t=0.33

In [154]:
invesment_tax_credit = []

for i in range(1,T1+1):    
    invesment_tax_credit.append( (0.5/T1)/((1+0.0808)**i) )                   
    #print(invesment_tax_credit, i)

invesment_tax_credit = sum(invesment_tax_credit)

In [155]:
invesment_tax_credit

0.283927520968014

In [156]:
depreciation = []
for j in range(1,T2+1):
    depreciation.append( (1/T2)*1.33/((1+0.0808)**j) )
    #print(depreciation, i)

depreciation = sum(depreciation)

In [157]:
ecu1 = 1-0.33*(depreciation + invesment_tax_credit)

In [158]:
ecu1

0.6128581822041248

In [159]:
incentivo_factor = ( 1/( 1-t ) )*ecu1

In [160]:
incentivo_factor

0.9147137047822759

In [161]:
1-(100/1000)**(1/25)

0.08798916064409024

# Levelised Cost of  Energy  (LCOE) 

In [162]:
p_load = financiero['p_load']
ens = financiero['ens']
lcoe = (asc/(p_load - ens))
lcoe_incentivo = (asc_incentivo/(p_load - ens))

NameError: name 'financiero' is not defined

In [24]:
p_load - ens

35689.85

In [25]:
sql_actualizar= """UPDATE parametros 
        SET  vida_proyecto = %s,
        ir = %s,
        crf= %s,
        cpv_ins= %s,
        cbat_ins= %s,
        cdg_ins= %s,
        capital_cpv= %s,
        capital_cbat= %s,
        capital_cdg= %s,
        kbat= %s,
        kdg= %s,
        ybat= %s,
        ydg= %s,
        rc_bat= %s,
        rc_dg= %s,
        factor_bat= %s,
        factor_pv= %s,
        oym_pv= %s,
        oym_bat= %s,
        ipp_actual= %s,
        trm= %s,
        pami = %s,
        plim = %s,
        oym_dg = %s,
        asc = %s,
        lcoe= %s,
        asc_incentivo=%s,
        lcoe_incentivo=%s,
        cost_e_dg=%s,
        cost_e_pv=%s,
        cost_e_bat=%s,
        cost_e_ens=%s        
        WHERE id_simulacion =%s"""%(R, 
                                    ir, 
                                    crf, 
                                    cpv_ins,
                                    cbat_ins, 
                                    cdg_ins, 
                                    ccpv ,
                                    ccbat,
                                    ccdg ,
                                    kbat, 
                                    kdg, 
                                    ybat, 
                                    ydg, 
                                    rc_bat, 
                                    rc_dg, 
                                    factor_bat,
                                    factor_pv, 
                                    round(oym_pv,2), 
                                    round(oym_bat,2), 
                                    round(ipp_actual,2), 
                                    round(trm,2), 
                                    round(pami,2), 
                                    round(plim,2),
                                    round(oym_dg,2), 
                                    round(asc,2), 
                                    round(lcoe,2), 
                                    round(asc_incentivo,2),
                                    round(lcoe_incentivo,2), 
                                    round(cost_e_dg,2),
                                    round(cost_e_pv,2) ,
                                    round(cost_e_bat,2),
                                    round(cost_e_ens,2),
                                    round(id_simulacion,2))

#conexion_bd(sql=sql_actualizar, update=True, task=actualizar)

In [26]:
print(sql_actualizar)

UPDATE parametros 
        SET  vida_proyecto = 20,
        ir = 0.0808,
        crf= 0.1,
        cpv_ins= 5605.365,
        cbat_ins= 539983.495,
        cdg_ins= 7627407.001,
        capital_cpv= 218609235.0,
        capital_cbat= 27215168.148,
        capital_cdg= 76274070.01,
        kbat= 0.6712,
        kdg= 0.6712,
        ybat= 0.7,
        ydg= 0.7,
        rc_bat= 12786774.6027,
        rc_dg= 35836609.0535,
        factor_bat= 0.02,
        factor_pv= 0.01,
        oym_pv= 2186092.35,
        oym_bat= 544303.36,
        ipp_actual= 129.23,
        trm= 3736.91,
        pami = 8789,
        plim = 79900,
        oym_dg = 15669183.32,
        asc = 14844.29,
        lcoe= 0.42,
        asc_incentivo=14211.46,
        lcoe_incentivo=0.4,
        cost_e_dg=3153.79,
        cost_e_pv=68.66,
        cost_e_bat=1364.06,
        cost_e_ens=318.79        
        WHERE id_simulacion =29


In [27]:
conexion_bd(sql=sql_actualizar)

In [1]:
import actualizar_costos as ac

In [2]:
parametros = ac.conexion_bd("SELECT * FROM parametros WHERE optimizacion = 'optimal'",  update=False)
resultados = ac.conexion_bd("SELECT * FROM resultados ", update=False)



In [3]:
for j in range(len(parametros)):
    ac.actualizar_costos(parametros=parametros, resultados=resultados, i=j)